# Add Flag and Emails New EFT Accounts

In [ ]:
import os
import time

%load_ext sql
%config SqlMagic.displaylimit = 5


This will create the connection to the database and prep the jupyter magic for SQL

In [ ]:
connect_to_db = 'postgresql://' + \
                os.getenv('PG_USER', '') + ":" + os.getenv('PG_PASSWORD', '') +'@' + \
                os.getenv('PG_HOST', '') + ':' + os.getenv('PG_PORT', '5432') + '/' + os.getenv('PG_DB_NAME', '');
    
%sql $connect_to_db


In [ ]:
%%sql stat_accounts_id  <<
SELECT pa.auth_account_id 
FROM payment_accounts pa
RIGHT JOIN invoices i ON i.payment_account_id = pa.id
WHERE i.payment_method_code IN ('EFT', 'DIRECT_PAY')
LIMIT 20;  # Limiting to 20 for the pilot run


In [ ]:
import requests
import base64

# Function to get environment or global variables (similar to Postman's pm.variables.get)
def get_var(variable_name, default=''):
    return os.getenv(variable_name, default)

headers = {
    'content-type': 'application/x-www-form-urlencoded',
    'Authorization': 'Basic ' + base64.b64encode(
        bytes(get_var('KEYCLOAK_SERVICE_ACCOUNT_ID') + ':' + get_var('KEYCLOAK_SERVICE_ACCOUNT_SECRET'),
        'utf-8')).decode('utf-8')
}
body = {
    'grant_type': 'client_credentials'
}
issuer_url = get_var('JWT_OIDC_ISSUER')
token_url =  issuer_url + '/protocol/openid-connect/token'
print(issuer_url)

# Send the request
response = requests.post(token_url, headers=headers, data=body)

# Check if the request was successful
if response.status_code == 200:
    json_response = response.json()
    new_access_token = json_response.get('access_token', '')

    # Set the new token to environment (or wherever you want to store it)
    os.environ['accessToken'] = new_access_token

    print(f"Successfully obtained access token: {new_access_token}")
else:
    print(f"Failed to obtain access token. Status code: {response.status_code}, Response: {response.text}")


In [ ]:
API_BASE_URL = os.getenv('PAY_API_URL', '')

headers = {
    "Authorization": f"Bearer {os.environ['accessToken']}",
    "Content-Type": "application/json"
}

successful = []
failed = []
# Loop through each auth_account_id and send a PATCH request
for auth_id in stat_accounts_id:
    url = f"{API_BASE_URL}/api/v1/accounts/{auth_id}/eft"
    response = requests.patch(url, headers=headers)
    
    # Check if the request was successful
    if response.status_code == 200:
        successful.push(auth_id)
    else:
        failed.push(auth_id)
        # Introduce a delay of 5 seconds between requests
    time.sleep(5)
